# Final Project: Stock Price vs Revenue Dashboard (Tesla & GameStop)

**Name:** _Your Name Here_

**Date:** 2025-09-01

---
This notebook completes the 7 questions specified in the assignment. It uses **yfinance** to pull stock prices and **web scraping** to collect quarterly revenue from Macrotrends. Finally, it builds dashboards comparing stock price to revenue for **Tesla (TSLA)** and **GameStop (GME)**.

> ⚠️ **Before you run this notebook**: Ensure you have completed the previous yfinance and web scraping labs and that your environment has internet access.

## What you'll submit
- **Screenshots** clearly showing each question header, the code you executed, and the resulting output/plots.
- **Shared copy** of this notebook (export to HTML or share via repo) for Question 7.

## Grading (12 points total)
- Q1 TSLA stock (2), Q2 TSLA revenue (1), Q3 GME stock (2), Q4 GME revenue (1)
- Q5 TSLA dashboard (2), Q6 GME dashboard (2), Q7 Share notebook (2)


## Setup
Install or import required libraries. If you're in a fresh environment, uncomment the `pip install` lines.

In [6]:

import pandas as pd
import numpy as np
import yfinance as yf
import requests
from bs4 import BeautifulSoup
import plotly.graph_objects as go
from plotly.subplots import make_subplots

pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 120)

---
## Helper: Clean Revenue Table from Macrotrends
Macrotrends pages contain multiple tables; we specifically want the **Quarterly Revenue** table.


In [7]:
def scrape_quarterly_revenue(macrotrends_url: str) -> pd.DataFrame:
    """Scrape Quarterly Revenue table (Date, Revenue) from a Macrotrends revenue page.
    Example URLs:
      Tesla:    https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue
      GameStop: https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue
    """
    headers = {"User-Agent": "Mozilla/5.0"}
    html = requests.get(macrotrends_url, headers=headers).text
    soup = BeautifulSoup(html, "html.parser")
    tables = soup.find_all("table")

    target_table = None
    for tbl in tables:
        caption = tbl.find_previous_sibling('h2')
        if caption and 'Quarterly Revenue' in caption.get_text(strip=True):
            target_table = tbl
            break

    if target_table is None:
        # Fallback: pick the first table that has 'Revenue' in header
        for tbl in tables:
            th = tbl.find('th')
            if th and 'Revenue' in th.get_text():
                target_table = tbl
                break

    if target_table is None:
        raise ValueError("Quarterly Revenue table not found on the page. Structure may have changed.")

    df = pd.read_html(str(target_table))[0]
    # Expect columns like ['Date', 'Revenue']
    df.columns = [c.strip() for c in df.columns]
    # Keep only 'Date' and 'Revenue' if present
    keep_cols = [c for c in df.columns if c.lower().startswith('date') or c.lower().startswith('revenue')]
    df = df[keep_cols]
    # Standardize column names
    rename_map = {c: 'Date' if 'date' in c.lower() else ('Revenue' if 'revenue' in c.lower() else c) for c in df.columns}
    df = df.rename(columns=rename_map)
    # Clean revenue string: remove $ and commas; coerce to numeric
    df['Revenue'] = (df['Revenue']
                     .astype(str)
                     .str.replace('$', '', regex=False)
                     .str.replace(',', '', regex=False)
                     .str.replace(' ', '', regex=False)
                    )
    df['Revenue'] = pd.to_numeric(df['Revenue'], errors='coerce')
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df = df.dropna(subset=['Date', 'Revenue']).sort_values('Date').reset_index(drop=True)
    return df

---
## Helper: Stock Downloader via yfinance


In [8]:
def download_stock_history(ticker: str, start: str = '2010-01-01') -> pd.DataFrame:
    """Download daily history with yfinance and keep Date, Close columns"""
    data = yf.download(ticker, start=start, progress=False)
    data = data[['Close']].dropna().reset_index()
    data.columns = ['Date', 'Close']
    return data

## Helper: Dashboard (Price vs Revenue)
Creates a 2-row interactive Plotly figure: stock price on top, quarterly revenue on bottom.


In [9]:
def price_vs_revenue_dashboard(price_df: pd.DataFrame, revenue_df: pd.DataFrame, title: str):
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=(f"{title} - Closing Price", f"{title} - Quarterly Revenue"))
    # Price trace
    fig.add_trace(
        go.Scatter(x=price_df['Date'], y=price_df['Close'], mode='lines', name='Close'),
        row=1, col=1
    )
    # Revenue trace
    fig.add_trace(
        go.Bar(x=revenue_df['Date'], y=revenue_df['Revenue'], name='Revenue'),
        row=2, col=1
    )
    fig.update_layout(height=700, width=1000, title_text=title, showlegend=True)
    fig.update_xaxes(title_text='Date', row=2, col=1)
    fig.update_yaxes(title_text='Price (USD)', row=1, col=1)
    fig.update_yaxes(title_text='Revenue (USD)', row=2, col=1)
    return fig

---
## Question 1 — Extracting Tesla Stock Data Using yfinance (2 pts)


In [10]:
tsla_price = download_stock_history('TSLA', start='2010-01-01')
print(tsla_price.head())
print(tsla_price.tail())
# Optional: save for reference
tsla_price.to_csv('tsla_price.csv', index=False)

C:\Users\Nihal Shaikh\AppData\Local\Temp\ipykernel_3480\3304570534.py:3: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, start=start, progress=False)


        Date     Close
0 2010-06-29  1.592667
1 2010-06-30  1.588667
2 2010-07-01  1.464000
3 2010-07-02  1.280000
4 2010-07-06  1.074000
           Date       Close
3812 2025-08-25  346.600006
3813 2025-08-26  351.670013
3814 2025-08-27  349.600006
3815 2025-08-28  345.980011
3816 2025-08-29  333.869995


## Question 2 — Extracting Tesla Revenue Data Using Web Scraping (1 pt)
We use Macrotrends' revenue page for Tesla.

In [11]:
tsla_rev_url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
tsla_revenue = scrape_quarterly_revenue(tsla_rev_url)
print(tsla_revenue.head())
print(tsla_revenue.tail())
tsla_revenue.to_csv('tsla_revenue.csv', index=False)

C:\Users\Nihal Shaikh\AppData\Local\Temp\ipykernel_3480\1699775314.py:30: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(target_table))[0]


KeyError: 'Revenue'

---
## Question 3 — Extracting GameStop Stock Data Using yfinance (2 pts)


In [ ]:
gme_price = download_stock_history('GME', start='2010-01-01')
print(gme_price.head())
print(gme_price.tail())
gme_price.to_csv('gme_price.csv', index=False)

## Question 4 — Extracting GameStop Revenue Data Using Web Scraping (1 pt)


In [ ]:
gme_rev_url = "https://www.macrotrends.net/stocks/charts/GME/gamestop/revenue"
gme_revenue = scrape_quarterly_revenue(gme_rev_url)
print(gme_revenue.head())
print(gme_revenue.tail())
gme_revenue.to_csv('gme_revenue.csv', index=False)

---
## Question 5 — Tesla Stock and Revenue Dashboard (2 pts)
Creates an interactive two-panel dashboard.

In [ ]:
fig_tsla = price_vs_revenue_dashboard(tsla_price, tsla_revenue, title='Tesla (TSLA) — Price vs Quarterly Revenue')
fig_tsla.show()
# Save a static image if you have kaleido installed:
# !pip install -U kaleido
# fig_tsla.write_image('tesla_dashboard.png')

## Question 6 — GameStop Stock and Revenue Dashboard (2 pts)


In [ ]:
fig_gme = price_vs_revenue_dashboard(gme_price, gme_revenue, title='GameStop (GME) — Price vs Quarterly Revenue')
fig_gme.show()
# Save a static image if you have kaleido installed:
# fig_gme.write_image('gamestop_dashboard.png')

---
## Question 7 — Sharing Your Assignment Notebook (2 pts)
Export this notebook to **HTML** and share the HTML or upload to a repository.

**How to export in Jupyter:**
- `File > Save and Checkpoint`
- `File > Download as > HTML (.html)`

**What to upload for grading:**
- Screenshots showing the code **and** outputs for Q1–Q6
- The shared notebook file (HTML or notebook link)


---
## (Optional) Sanity Checks & Alignment of Frequencies
Price is **daily** while revenue is **quarterly**. For plotting they are shown on the same x-axis but different frequencies. If you need to align them explicitly, this helper aggregates price to quarter-end.


In [ ]:
def to_quarter_end(df_price: pd.DataFrame) -> pd.DataFrame:
    q = (df_price
         .set_index('Date')['Close']
         .resample('Q')
         .last()
         .reset_index())
    q.columns = ['Date', 'Close']
    return q

# Example usage (optional):
# tsla_price_q = to_quarter_end(tsla_price)
# gme_price_q  = to_quarter_end(gme_price)

---
## (Optional) Save Clean Datasets
This saves tidy CSVs you can reuse or turn in alongside screenshots.


In [ ]:
# Already saved above; you can re-run anytime:
# tsla_price.to_csv('tsla_price.csv', index=False)
# tsla_revenue.to_csv('tsla_revenue.csv', index=False)
# gme_price.to_csv('gme_price.csv', index=False)
# gme_revenue.to_csv('gme_revenue.csv', index=False)

---
## (Optional) Peer Review Checklist (for grading a peer)
- [ ] **Q1:** yfinance used for TSLA; dataframe printed; looks correct
- [ ] **Q2:** Web scraping used for TSLA revenue; columns cleaned to numeric; dates parsed
- [ ] **Q3:** yfinance used for GME; dataframe printed; looks correct
- [ ] **Q4:** Web scraping used for GME revenue; columns cleaned to numeric; dates parsed
- [ ] **Q5:** Dashboard shows TSLA price (line) and revenue (bar) with appropriate labels
- [ ] **Q6:** Dashboard shows GME price (line) and revenue (bar) with appropriate labels
- [ ] **Q7:** Notebook shared (HTML or link); screenshots clearly show code **and** outputs
- [ ] Code quality: clear functions, comments, no hard-coded fragile selectors
- [ ] Data quality: no obvious NaNs in key columns; sorted by Date
- [ ] Visual quality: axes labeled, titles present


In [ ]:
pip install yfinance requests bs4 plotly pandas numpy